# Email Service - Send to all consumers

### Download dependencies

In [2]:
%pip install openai bs4 pandas requests\

!apt-get update
!apt install chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://cli.github.com/packages stable InRelease                         
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease    
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,290 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,539 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2

### Create Zero Shot prompt
This will be used to guide LLM when classifying text into the 5 political concepts

In [3]:
text = """
# Identity

You are AI model that classifies text into one of the 4 distinct categories. You will be given text as input. That is what you need to classify into one of the categories. 

The 5 categories are **Immigration**, **Economy**, **Healthcare**, **Civil Rights**, **Education**

# Instructions

* Do not answer in a sentence at all. 
* Do not give responses with Markdown formatting, just return a one word answer which corresponds to one of the 5 categories mentioned
* Never answer in a sentence. 
"""

with open('political_text_classifier.txt', 'w') as f:
    f.write(text)


### Load Zero Shot prompt
This is needed in the future (once webscraper is fully ran)

In [4]:
with open("political_text_classifier.txt", "r", encoding="utf-8") as file:
    instructions = file.read()

### Define function to run the political text classifer
This uses OpenAI SDK and GPT 4o as base LLM

In [ ]:
from openai import OpenAI
client = OpenAI()

def classifyText(input_text: str):
    response = client.chat.completions.create(
        model="gpt-4o",
        instructions=instructions,
        input=input_text
    )

    return response

### Run webscraper to get NYC legislation
Beautifulsoup will be used to access HTML of NYC Legistar website. We will save the scrapped data in a Pandas dataframe for further processing

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

request_url = "https://legistar.council.nyc.gov/Calendar.aspx?Mode=This+Week"
web_html = requests.get(request_url).text
soup = BeautifulSoup(web_html, "html.parser")
table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridCalendar_ctl00')

rows = []

# Skip the header row which is index 0
# Only get the 3 most recent meetings
for tr in table.find_all('tr')[1:4]:  
    cells = tr.find_all('td')
    date = cells[1].get_text(strip=True)
    committee = cells[0].get_text(strip=True)
        
    # Get the agenda link (third column - find <a> within <span>)
    meeting_detail = cells[6].find('a')
    meeting_detail_aspx = meeting_detail['href']    
    
    rows.append({
            'Date': date,
            'Committee': committee,
            'Meeting Details': meeting_detail_aspx
    })


### View council meetings in a formatted manner
This is accomplished through Pandas

In [9]:
import pandas as pd

df = pd.DataFrame(rows, columns=rows[0])
df

,Date,Committee,Meeting Details
0,11/25/2025,City Council Stated Meeting,MeetingDetail.aspx?ID=1359999&GUID=80C2CF9F-9E...
1,11/25/2025,Committee on Women and Gender Equity,MeetingDetail.aspx?ID=1355584&GUID=629C1A78-9F...
2,11/25/2025,Committee on General Welfare,MeetingDetail.aspx?ID=1356244&GUID=6AA28AE4-D9...


### Scrap Meeting Details page
Iterate through each meeting detail and iterate through each "Introduction" legislative text